In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import joblib

In [14]:
#importing the dataset
dataset = pd.read_csv("phishcoop.csv")
dataset = dataset.drop('id', 1) #removing unwanted column

<ipython-input-14-cfd12b8d8270>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dataset = dataset.drop('id', 1) #removing unwanted column


In [15]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6992 entries, 0 to 6991
Data columns (total 31 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   having_IP_Address            6992 non-null   int64
 1   URL_Length                   6992 non-null   int64
 2   Shortining_Service           6992 non-null   int64
 3   having_At_Symbol             6992 non-null   int64
 4   double_slash_redirecting     6992 non-null   int64
 5   Prefix_Suffix                6992 non-null   int64
 6   having_Sub_Domain            6992 non-null   int64
 7   SSLfinal_State               6992 non-null   int64
 8   Domain_registeration_length  6992 non-null   int64
 9   Favicon                      6992 non-null   int64
 10  port                         6992 non-null   int64
 11  HTTPS_token                  6992 non-null   int64
 12  Request_URL                  6992 non-null   int64
 13  URL_of_Anchor                6992 non-null   int

In [16]:
x = dataset.iloc[ : , :-1].values
y = dataset.iloc[:, -1:].values

In [17]:
#spliting the dataset into training set and test set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.25, random_state =0 )

In [18]:
#----------------applying grid search to find best performing parameters 
from sklearn.model_selection import GridSearchCV
parameters = [{'n_estimators': [100, 700],
    'max_features': ['sqrt', 'log2'],
    'criterion' :['gini', 'entropy']}]

In [19]:
grid_search = GridSearchCV(RandomForestClassifier(),  parameters,cv =5, n_jobs= -1)
grid_search.fit(x_train, y_train)
#printing best parameters 
print("Best Accurancy =" +str( grid_search.best_score_))
print("best parameters =" + str(grid_search.best_params_))

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


Best Accurancy =0.9704416783705309
best parameters ={'criterion': 'gini', 'max_features': 'sqrt', 'n_estimators': 700}


In [20]:
#fitting RandomForest regression with best params 
classifier = RandomForestClassifier(n_estimators = 100, criterion = "gini", max_features = 'log2',  random_state = 0)
classifier.fit(x_train, y_train)

#predicting the tests set result
y_pred = classifier.predict(x_test)

#confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

<ipython-input-20-8346b2ed7403>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(x_train, y_train)


In [21]:
print(cm)


[[808  20]
 [ 21 899]]


In [23]:
#pickle file joblib
joblib.dump(classifier, 'rf_final.pkl')


['rf_final.pkl']

In [24]:
#-------------Features Importance random forest
names = dataset.iloc[:,:-1].columns
importances =classifier.feature_importances_
sorted_importances = sorted(importances, reverse=True)
indices = np.argsort(-importances)
var_imp = pd.DataFrame(sorted_importances, names[indices], columns=['importance'])

